# VacationPy
----

## Overview

* Uses the csv dataset produced from the WeatherPy code to search for infomation on the Google Maps API.

* Includes a heat map using latitude and longitute values as weights

* Includes a map with markers showing Hotel name, city, and country for a subset of areas that were within ideal weather conditions. 


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create dataframe based on saved csv data from WeatherPy file

vacation_df = pd.read_csv("output_data/cities.csv")

# Display top 5 rows of dataframe
vacation_df.head()


,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,provideniya,64.3833,-173.3000,-7.28,82,92,7.67,RU,1611956247
1,1,touros,-5.1989,-35.4608,80.60,74,20,12.66,BR,1611956296
2,2,rikitea,-23.1203,-134.9692,80.11,67,88,7.54,PF,1611956297
3,3,salalah,17.0151,54.0924,73.40,60,75,5.75,OM,1611956297
4,4,cape town,-33.9258,18.4232,69.01,82,20,9.22,ZA,1611956138


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps

gmaps.configure(api_key=g_key)

In [4]:
# Heatmap using lat and lng as weights

locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

# Note - manually zoom in jupyter notebook to see map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Set restrictions for ideal weather conditions
# Max temp less than 80 and over 70, and wind speed less than 10, and 0% cloudy
# Drop missing values

vacation_ideal_df = vacation_df.loc[
    (vacation_df["Max Temp"] < 80) &
    (vacation_df["Max Temp"] > 70) &
    (vacation_df["Wind Speed"] < 10) &
    (vacation_df["Cloudiness"] == 0)
].dropna()

# Display dataframe top 5 rows
vacation_ideal_df.head()

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,saint-philippe,-21.3585,55.7679,78.80,65,0,5.75,RE,1611956219
119,119,camana,-16.6228,-72.7111,73.65,66,0,6.76,PE,1611956322
239,239,tokonou,9.6500,-9.7833,72.55,20,0,2.48,GN,1611956349
294,294,mopti,14.6667,-4.0000,77.00,29,0,2.30,ML,1611956361
304,304,grand gaube,-20.0064,57.6608,78.01,84,0,4.00,MU,1611956363


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create copy of hotel_df dataframe, extract columns for city, country, and coordinates
hotel_df = vacation_ideal_df[["City", "Country", "Lat", "Lng"]].copy()

# Add Hotel Name column to hotel_df
hotel_df["Hotel Name"] = ""

# Display dataframe top 5 rows
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
47,saint-philippe,RE,-21.3585,55.7679,
119,camana,PE,-16.6228,-72.7111,
239,tokonou,GN,9.6500,-9.7833,
294,mopti,ML,14.6667,-4.0000,
304,grand gaube,MU,-20.0064,57.6608,


In [7]:
# Search paramaters for url

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through rows and columns
for hotel, row in hotel_df.iterrows():
    
    # Save lat and lng from dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    
    # params with location for url
    params["location"] = f"{lat},{lng}"
    
    # Using search for the term Hotel
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Request to url
    response_json = requests.get(base_url, params=params).json()
    
    # Check when reach 8
    if hotel == 8:
        pprint(response_json)

    
    # Store name of hotel in dataframe
    try:
        hotel_df.loc[hotel, "Hotel Name"] = response_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing, skipping...")

        
# Display dataframe with hotel names
hotel_df


Missing, skipping...
Missing, skipping...
Missing, skipping...
Missing, skipping...


,City,Country,Lat,Lng,Hotel Name
47,saint-philippe,RE,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
119,camana,PE,-16.6228,-72.7111,Hostal Morales
239,tokonou,GN,9.6500,-9.7833,
294,mopti,ML,14.6667,-4.0000,
304,grand gaube,MU,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
348,saint-pierre,RE,-21.3393,55.4781,Lindsey Hôtel
384,koundara,GN,12.4833,-13.3000,
431,yaan,NG,7.3833,8.5667,
442,oussouye,SN,12.4850,-16.5469,Auberge du Routard
481,caravelas,BR,-17.7125,-39.2481,Pousada dos Navegantes


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))